In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段3：形式化：使用Vertex AI模型注册表开始
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage3/get_started_with_model_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>         
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用 Vertex AI SDK 使用 `Vertex AI 模型注册表` 创建和注册多个模型版本。

### 目标

在本教程中，您将学习如何使用`Vertex AI模型注册表`来创建和注册模型的多个版本。

本教程使用以下谷歌云ML服务和资源:

- `Vertex AI模型注册表`
- `Vertex AI模型`资源
- `Google Cloud管道组件`

执行的步骤包括:

- 创建并注册模型的第一个版本到`Vertex AI模型注册表`。
- 创建并注册模型的第二个版本到`Vertex AI模型注册表`。
- 更新默认的模型版本（被批准的）。
- 删除一个模型版本。
- 重新训练下一个模型版本。

数据集

本教程使用了一个在ImageNet数据集上训练过的预训练图像分类模型，来自于TensorFlow Hub。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 基于您的预期使用情况生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

### 重启内核

安装额外包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果有可能，请确保您在 GPU 运行时中运行本笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您会获得 $300 的免费信用用于支付计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs、Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并插入以 `$` 为前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。 以下是Vertex AI支持的地区。 我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。 并非所有地区都支持所有Vertex AI服务。

了解有关[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在一个直播教程会话中，您可能会使用一个共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经通过验证。请跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth认证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**栏中，输入一个名称，然后点击**创建**。

在**将此服务账户授权访问项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中输入您服务账户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储临时存储桶。临时存储桶是您的数据集和模型资源之间的所有数据在会话之间保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在你的存储桶尚不存在时才能运行以下单元格来创建你的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器镜像，并指定分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器镜像，为每个VM分配4个Nvidia Telsa K80 GPU，则应指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定`(None, None)`以使用一个容器镜像在CPU上运行。

了解有关[您所在地区硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的更多信息。

*注意*：TF 2.3之前的GPU支持版本无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由于生成在serving函数中的静态图操作。如果在自定义模型中遇到此问题，请使用带有GPU支持的TF 2.3容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预先构建的容器

为预测设置预先构建的Docker容器映像。

有关最新列表，请参阅[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个CPU核心

*注意：您也可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

## 模型注册表介绍

`Vertex AI模型注册表`服务为您提供了创建和分组多个版本的`模型`资源的能力。通过版本管理，您可以：

- 跨版本跟踪模型传承
- 设置哪个版本是默认版本 -- 也就是被认可的版本。
- 其他服务操作，如部署，将自动使用默认版本 -- 而无需明确指定版本。
- 为各个版本分配别名 -- 如开发、测试和生产。

了解更多关于[模型注册表介绍](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。

## 从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的两个预训练模型版本，然后将它们上传到`Vertex AI Model`资源中作为版本1和版本2。一旦您拥有了`Vertex AI Model`资源，模型版本就可以部署到`Vertex AI Endpoint`资源中。

### 下载预训练模型

首先，您要从TensorFlow Hub下载两个版本的预训练模型。这些模型以TF.Keras层的形式下载。在本示例中，为了完成模型，您将创建一个带有下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model_v1 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v1.build([None, 224, 224, 3])

tfhub_model_v1.summary()

In [ ]:
tfhub_model_v2 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v2.build([None, 128, 128, 3])

tfhub_model_v2.summary()

### 保存模型文件

此时，两个版本的模型都在内存中。接下来，您将模型文件保存到云存储位置。

In [ ]:
MODEL_V1_DIR = BUCKET_URI + "/model/v1"
tfhub_model_v1.save(MODEL_V1_DIR)

MODEL_V2_DIR = BUCKET_URI + "/model/v2"
tfhub_model_v2.save(MODEL_V2_DIR)

### 将 TensorFlow Hub 模型的版本1上传到 `Vertex AI Model` 资源

接下来，您将第一个版本的模型（`MODEL_DIR_V1`）作为 `Model` 资源上传到 `Model Registry` 中，同时附加以下参数：

- `is_default_version`：指定这是否将成为 `Model` 资源的默认版本。
- `version_ailiases`：用户定义的模型版本的替代别名列表，例如 `production`。
- `version_description`：模型版本的用户描述。

当在 `Model Registry` 中创建第一个模型版本时，属性 `version_id` 将自动设置为1。

In [ ]:
model_v1 = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_V1_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1", "v.1", "1"],
    version_description="This is the first version of the model",
)

print(model_v1)

### 上传 TensorFlow Hub 模型的第二个版本到 `Vertex AI Model` 资源

然后，您将第二个版本的模型 (`MODEL_DIR_V1`) 作为 `Model` 资源上传到 `Model Registry`，并添加以下附加参数：

- `parent_model`：要将此模型添加为下一个模型版本的现有 `Model` 资源。
- `is_default_version`：指示此模型是否将成为 `Model` 资源的默认版本。在此示例中，您将从第一个版本的默认版本更改为第二个版本。
- `version_ailiases`：该模型版本的用户自定义备用别名列表，如 `production`。
- `version_description`：模型版本的用户描述。

在 `Model Registry` 中创建后续模型版本时，属性 `version_id` 将自动递增。在此示例中，将设置为 2（第二个版本）。

In [ ]:
model_v2 = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_V2_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=model_v1.resource_name,
    is_default_version=True,
    version_aliases=["v2", "v.2", "2"],
    version_description="This is the second version of the model",
)

print(model_v2)

获取父模型的所有版本

接下来，您可以使用`version_registry.list_versions()`列出父模型的所有版本。

In [ ]:
versions = model_v1.versioning_registry.list_versions()
for version in versions:
    print(version)

获取非父模型的所有版本

您重复相同的操作，但这次使用非父模型。如您所看到的，您会获取所有版本，无论您指定的模型是父模型还是非父模型。

In [ ]:
versions = model_v2.versioning_registry.list_versions()
for version in versions:
    print(version)

列出一个`Model`资源

与`Model`资源相关的其余`Vertex AI` SDK方法将自动使用`Model`资源的默认版本。

接下来，您可以使用带有过滤器的`list()`方法来获取您上面创建的`Model`资源。在这个例子中，有两个版本。版本2被设置为默认版本，因此list()只会返回版本2（默认版本）的信息。

In [ ]:
models = aip.Model.list(filter=f"display_name=example_{TIMESTAMP}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

### 修改默认模型版本

接下来，您可以使用`versioning_registry.add_version_aliases()`方法来修改`Model`资源的默认模型版本，具体操作如下：

- `version`：该操作适用于指定版本ID的哪个版本。
- `new_aliases`：要分配给指定版本的附加别名。

在本例中，您可以引用特殊别名`default`来将此模型（版本1）更改为默认版本。

接下来，您将使用`list()`方法，该方法现在将返回模型的第1版本，而不是第2版本。

In [ ]:
model_v2.versioning_registry.add_version_aliases(new_aliases=["default"], version="1")

models = aip.Model.list(filter=f"display_name=example_{TIMESTAMP}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建一个`Endpoint`资源。至少，您需要指定端点的显示名称。可选地，您还可以指定项目和位置（区域）；否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的可读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）以键值对形式为`Endpoint`指定的用户定义的元数据。

该方法将返回一个`Endpoint`对象。

了解有关 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api) 的更多信息。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个终端点。每个部署的`Vertex AI Model`资源将拥有自己的用于提供二进制文件的部署容器。

*备注:* 在上传模型工件到`Vertex AI Model`资源的上一步中，您为TFHub模型指定了部署容器。

### 部署单个`Endpoint`资源

在下一个示例中，您将将`Vertex AI Model`资源的默认版本部署到`Vertex AI Endpoint`资源。`Vertex AI Model`资源已经为其定义了部署容器镜像。要进行部署，您需指定以下附加配置设置：

- 机器类型。
- GPU的类型和数量（如果有）。
- VM实例的静态、手动或自动扩展。

在此示例中，您将以最少的指定参数部署模型，具体如下：

- `model`: `Model`资源。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `machine_type`: 每个VM实例的机器类型。

由于需要配置资源，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

获取部署模型信息

您可以从`Endpoint`资源配置数据`gca_resource.deployed_models`中获取部署模型的部署设置。在此示例中，只部署了一个模型，因此引用了下标`[0]`。

In [ ]:
endpoint.gca_resource.deployed_models[0]

### 从 `Endpoint` 资源中取消部署 `Model` 资源

当一个 `Model` 资源被部署到一个 `Endpoint` 资源时，被部署的 `Model` 资源实例会被赋予一个ID -- 通常被称为部署模型的ID。

您可以使用 `undeploy()` 方法取消部署特定的 `Model` 资源实例，参数如下：

- `deployed_model_id`：分配给部署模型的ID。

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

### 删除模型版本

要从“模型注册表”中删除模型的版本，您可以使用`versioning_registry.delete_version()` 方法，并使用以下参数：

- `version`：可以是`version_id` 或来自`version_alias` 的别名。

*注意：* 您无法删除默认模型版本。

In [ ]:
# Deletes a specific model version, as long as it isn't the default version.
model.versioning_registry.delete_version("2")

versions = model.versioning_registry.list_versions()
for version in versions:
    print(version)

删除`Model`资源时，所有相关的版本都会被删除。

In [ ]:
model.delete()

### 为`Vertex AI Training`指定模型版本

在使用`Vertex AI Training`服务训练模型时，您可以通过以下额外的参数指定训练后的模型为`Model Registry`中的父模型或父模型的一个版本，这些参数将传递给`run()`方法：

- `parent_model`：要将此模型添加为下一个模型版本的现有`Model`资源。
- `is_default_version`：此版本是否将成为`Model`资源的默认版本。
- `model_version_aliases`：用户定义的替代别名列表，用于模型版本，例如`production`。
- `model_version_description`：模型版本的用户描述。

以下是一个示例用法：

```
# 此作业将上传my-training-job模型的新的非默认版本
job = aiplatform.CustomTrainingJob(
   display_name="my-training-job",
   script_path="training_script.py",
   container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-2:latest",
   requirements=["gcsfs==0.7.1"],
   model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-2:latest",
)

model = job.run(
   my_dataset,
   replica_count=1,
   machine_type="n1-standard-4",
   accelerator_type='NVIDIA_TESLA_K80',
   accelerator_count=1,
   model_id='my-model'
   model_version_aliases=["myalias"],
   model_version_description="My description",
   is_default_version=False,
)

```

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您在教程中使用的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}